In [1]:
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
random_seed = 42
learning_rate = 0.001
batch_size = 64
n_epochs = 15

img_size = 224
n_classes = 10

In [84]:
def get_accuracy(model, data_loader, device):
  correct_pred = 0
  n = 0

  with torch.no_grad():
    model.eval()
    for X, y_true in data_loader:
      X, y_true = X.to(device), y_true.to(device)

      y_pred = model(X)
      _, predicted_labels = torch.max(y_pred, 1)

      n += y_true.size(0)
      correct_pred += (predicted_labels == y_true).sum()
  
  return correct_pred.float() / n

In [70]:
def train(train_loader, model, criterion, optimizer, device):
  model.train()
  running_loss = 0
  for X, y_true in train_loader:
    optimizer.zero_grad()

    X, y_true = X.to(device), y_true.to(device)
    y_hat = model(X)
    loss = criterion(y_hat, y_true)
    running_loss += loss.item() * X.size(0)
    loss.backward()
    optimizer.step()

  epoch_loss = running_loss / len(train_loader.dataset)
  return model, optimizer, epoch_loss

In [71]:
def validation(valid_loader, model, criterion, device):
  model.eval()
  running_loss = 0
  for X, y_true in valid_loader:
    X, y_true = X.to(device), y_true.to(device)

    y_hat= model(X)
    loss = criterion(y_hat, y_true)
    running_loss += loss.item() * X.size(0)
  
  epoch_loss = running_loss / len(valid_loader.dataset)

  return model, epoch_loss

In [72]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every = 1):
  best_loss = 1e10
  train_losses = []
  valid_losses = []

  for epoch in range(epochs):
    model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
    train_losses.append(train_loss)

    with torch.no_grad():
      model, valid_loss = validation(valid_loader, model, criterion, device)
      valid_losses.append(valid_loss)

    if epoch % print_every == (print_every - 1):
      train_acc = get_accuracy(model, train_loader, device)
      valid_acc = get_accuracy(model, valid_loader, device)
            
      print(f'{datetime.now().time().replace(microsecond=0)} --- '
            f'Epoch: {epoch}\t'
            f'Train loss: {train_loss:.4f}\t'
            f'Valid loss: {valid_loss:.4f}\t'
            f'Train accuracy: {100 * train_acc:.2f}\t'
            f'Valid accuracy: {100 * valid_acc:.2f}')
          
  return model, optimizer, (train_losses, valid_losses)      

In [73]:
common_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(227)
])

In [74]:
train_dataset = datasets.CIFAR10(root = 'cifar10',
                                 train = True,
                                 transform = common_transforms,
                                 download = True)

test_dataset = datasets.CIFAR10(root = 'cifar10',
                                train = False,
                                transform = common_transforms,
                                download = True)

Files already downloaded and verified
Files already downloaded and verified


In [75]:
import numpy as np

meanRGB = [np.mean(x.numpy(), axis = (1, 2)) for x, _ in train_dataset]
stdRGB = [np.std(x.numpy(), axis = (1, 2)) for x, _ in train_dataset]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print(meanR, meanG, meanB)
print(stdR, stdG, stdB)

0.49140054 0.4821596 0.4465322
0.19510235 0.19232473 0.19405492


In [76]:
train_transformer = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((227, 227)),
                transforms.RandomHorizontalFlip(),
                transforms.Normalize([0.49139965, 0.48215845, 0.4465309],
                                     [0.20220213, 0.19931543, 0.20086348])
])

test_transformer = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((227, 227)),
                transforms.Normalize([0.49139965, 0.48215845, 0.4465309],
                                     [0.20220213, 0.19931543, 0.20086348])
])

In [77]:
train_dataset.transforms = train_transformer
test_dataset.transforms = test_transformer

In [78]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 0)
indices = list(range(len(test_dataset)))
y_test0 = [y for _, y in test_dataset]

for test_index, val_index in sss.split(indices, y_test0):
    print('test :', len(test_index) , 'val :', len(val_index))

test : 8000 val : 2000


In [79]:
from torch.utils.data import Subset

valid_dataset = Subset(test_dataset, val_index)
test_dataset = Subset(test_dataset, test_index)

In [80]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [81]:
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
  def __init__(self, n_classes):
    super(AlexNet, self).__init__()
    
    self.feature_extractor = nn.Sequential(
      # First Layer
      nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4),
      nn.ReLU(inplace = True),
      nn.LocalResponseNorm(size = 5, alpha = 1e-3, beta = 0.75, k = 2),
      nn.MaxPool2d(kernel_size = 3, stride = 2),

      # Second Layer
      nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5,stride = 1, padding = 2),
      nn.ReLU(),
      nn.LocalResponseNorm(size = 5, alpha = 1e-4, beta = 0.75, k = 2),
      nn.MaxPool2d(kernel_size = 3, stride = 2),

      # Third Layer
      nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),

      # Fourth Layer
      nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),

      nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(3, 2),
    )

    self.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(256 * 6 * 6, out_features = 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features = 4096, out_features = 4096),
        nn.ReLU(),
        nn.Linear(in_features = 4096, out_features = n_classes),
    )

  def init_weight(self):
    for layer in self.feature_extractor:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean = 0, std = 0.01)
        nn.init.constant_(layer.bias, 0)
    nn.init.constant_(self.net[4].bias, 1)
    nn.init.constant_(self.net[10].bias, 1)
    nn.init.constant_(self.net[12].bias, 1)
  
  def forward(self , x):
    x = self.feature_extractor(x)
    x = x.view(-1, 256*6*6)
    x = self.classifier(x)

    return x

In [82]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [85]:
model = AlexNet(10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()

model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, 15, device)

12:23:04 --- Epoch: 0	Train loss: 1.8347	Valid loss: 1.5417	Train accuracy: 42.92	Valid accuracy: 42.70
12:25:24 --- Epoch: 1	Train loss: 1.4907	Valid loss: 1.3693	Train accuracy: 51.39	Valid accuracy: 50.70
12:27:44 --- Epoch: 2	Train loss: 1.3357	Valid loss: 1.2783	Train accuracy: 57.36	Valid accuracy: 54.50
12:30:06 --- Epoch: 3	Train loss: 1.2157	Valid loss: 1.1371	Train accuracy: 63.58	Valid accuracy: 60.25
12:32:27 --- Epoch: 4	Train loss: 1.1159	Valid loss: 1.0581	Train accuracy: 67.13	Valid accuracy: 63.30
12:34:47 --- Epoch: 5	Train loss: 1.0466	Valid loss: 0.9996	Train accuracy: 70.32	Valid accuracy: 64.75
12:37:07 --- Epoch: 6	Train loss: 0.9750	Valid loss: 1.0412	Train accuracy: 69.75	Valid accuracy: 63.20
12:39:29 --- Epoch: 7	Train loss: 0.9231	Valid loss: 0.9490	Train accuracy: 74.08	Valid accuracy: 66.90
12:41:50 --- Epoch: 8	Train loss: 0.8834	Valid loss: 0.9235	Train accuracy: 76.13	Valid accuracy: 68.50
12:44:10 --- Epoch: 9	Train loss: 0.8471	Valid loss: 0.8698	Trai

In [35]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
from torchsummary import summary

In [40]:
summary(model, input_size = (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
 LocalResponseNorm-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
 LocalResponseNorm-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,